In [1]:
#initial assumptions/settings - 
#team in "top" position will be considered home team

In [2]:
REPS = 100
STRAT = 'sim'

In [3]:
import pandas as pd
import pickle
import os
from tqdm import tqdm
from helper_functions.simulate_game import simulate_game
from helper_functions.combine_matrices import combine_team_matrix
from helper_functions.tourney_sim import matchup, nn_output
import numpy as np

In [82]:
#play-in
team_A = 'Virginia'
team_B = 'Colorado St.'

res=matchup(team_A, team_B, 'nn1', reps=5000, argmax=False)
res2=matchup(team_B, team_A, 'nn1', reps=5000, argmax=False)
l1=[x[0]>x[1] for x in res]
l2=[x[0]<x[1] for x in res2]
pd.Series(l1+l2).value_counts()

False    7082
True     2918
Name: count, dtype: int64

In [68]:
#play-in
team_A = 'Grambling'
team_B = 'Montana St.'

matrix_A = pd.read_excel('team_specific_matrix/{}_A.xlsx'.format(team_A), index_col='Starting_State')
matrix_B = pd.read_excel('team_specific_matrix/{}_B.xlsx'.format(team_B), index_col='Starting_State')

transition_matrix = combine_team_matrix(matrix_A, matrix_B)
res = simulate_game(transition_matrix, 500)

howard_win_list_1 = [x[0]>x[1] for x in res[1]]

#play-in
team_B = 'UConn'
team_A = 'Stetson'

matrix_A = pd.read_excel('team_specific_matrix/{}_A.xlsx'.format(team_A), index_col='Starting_State')
matrix_B = pd.read_excel('team_specific_matrix/{}_B.xlsx'.format(team_B), index_col='Starting_State')

transition_matrix = combine_team_matrix(matrix_A, matrix_B)
res = simulate_game(transition_matrix, 500)

howard_win_list_2 = [x[0]<x[1] for x in res[1]]

In [69]:
pd.Series(howard_win_list_1+howard_win_list_2).value_counts()

False    545
True     455
Name: count, dtype: int64

In [114]:
with open('models/nn1.pickle', 'rb') as file:
    nn1 = pickle.load(file)

In [ ]:
#sim model says wagner wins with 59% probability

In [75]:
te1='UConn'
te2='Iowa St.'
te3='Illinois'
te4='Auburn'
te5='San Diego St.'
te6='BYU'
te7='Washington St.'
te8='Fla. Atlantic'
te9='Northwestern'
te10='Drake'
te11='Duquesne'
te12='UAB'
te13='Yale'
te14='Morehead St.'
te15='South Dakota St.'
te16='Stetson'

tw1='North Carolina'
tw2='Arizona'
tw3='Baylor'
tw4='Alabama'
tw5="Saint Mary's (CA)"
tw6='Clemson'
tw7='Dayton'
tw8='Mississippi St.'
tw9='Michigan St.'
tw10='Nevada'
tw11='New Mexico'
tw12 = 'Grand Canyon'
tw13='Col. of Charleston'
tw14='Colgate'
tw15='Long Beach St.'
tw16='Howard'

ts1='Houston'
ts2 = 'Marquette'
ts3='Kentucky'
ts4='Duke'
ts5='Wisconsin'
ts6='Texas Tech'
ts7='Florida'
ts8='Nebraska'
ts9='Texas A&M'
ts10='Boise St.'
ts11='NC State'
ts12='James Madison'
ts13='Vermont'
ts14 = 'Oakland'
ts15='Western Ky.'
ts16='Longwood'

tm1='Purdue'
tm2='Tennessee'
tm3='Creighton'
tm4='Kansas'
tm5='Gonzaga'
tm6='South Carolina'
tm7='Texas'
tm8='Utah St.'
tm9='TCU'
tm10='Virginia'
tm11='Oregon'
tm12='McNeese'
tm13='Samford'
tm14='Akron'
tm15="Saint Peter's"
tm16='Grambling'

In [76]:
eastern_bracket = [(te1,te16),
                    (te8,te9),
                    (te5,te12),
                    (te4,te13),
                    (te6,te11),
                    (te3,te14),
                    (te7,te10),
                    (te2,te15)]

western_bracket = [(tw1,tw16),
                    (tw8,tw9),
                    (tw5,tw12),
                    (tw4,tw13),
                    (tw6,tw11),
                    (tw3,tw14),
                    (tw7,tw10),
                    (tw2,tw15)]

southern_bracket = [(ts1,ts16),
                    (ts8,ts9),
                    (ts5,ts12),
                    (ts4,ts13),
                    (ts6,ts11),
                    (ts3,ts14),
                    (ts7,ts10),
                    (ts2,ts15)]

midwestern_bracket = [(tm1,tm16),
                    (tm8,tm9),
                    (tm5,tm12),
                    (tm4,tm13),
                    (tm6,tm11),
                    (tm3,tm14),
                    (tm7,tm10),
                    (tm2,tm15)]

In [77]:
first_round = [eastern_bracket, western_bracket, southern_bracket, midwestern_bracket]

In [78]:
#RUN FIRST ROUND
#try with 10 simulations
east_round_2 = []
west_round_2 = []
south_round_2 = []
midwest_round_2 = []
second_round = [east_round_2,west_round_2,south_round_2,midwest_round_2]

for division_number in range(len(first_round)):
    for match in first_round[division_number]:
        print('Matchup: {} vs. {}'.format(match[0], match[1]))
        res=matchup(match[0],match[1], reps=REPS)
        winners = np.mean([x[0]>x[1] for x in res])
        if winners == 0.5:
            res=matchup(match[0],match[1])
            winners=res[0][0]>res[0][1]
        if winners > 0.5:
            print('{} wins'.format(match[0]))
            second_round[division_number].append(match[0])
        else:
            print('{} wins'.format(match[1]))
            second_round[division_number].append(match[1])
        print()
    print('-------------------------------------------------------------------------')
        



Matchup: UConn vs. Stetson
UConn wins

Matchup: Fla. Atlantic vs. Northwestern
Fla. Atlantic wins

Matchup: San Diego St. vs. UAB
San Diego St. wins

Matchup: Auburn vs. Yale
Yale wins

Matchup: BYU vs. Duquesne
BYU wins

Matchup: Illinois vs. Morehead St.
Morehead St. wins

Matchup: Washington St. vs. Drake
Drake wins

Matchup: Iowa St. vs. South Dakota St.
Iowa St. wins

-------------------------------------------------------------------------
Matchup: North Carolina vs. Howard
North Carolina wins

Matchup: Mississippi St. vs. Michigan St.
Mississippi St. wins

Matchup: Saint Mary's (CA) vs. Grand Canyon
Grand Canyon wins

Matchup: Alabama vs. Col. of Charleston
Col. of Charleston wins

Matchup: Clemson vs. New Mexico
Clemson wins

Matchup: Baylor vs. Colgate
Colgate wins

Matchup: Dayton vs. Nevada
Dayton wins

Matchup: Arizona vs. Long Beach St.
Arizona wins

-------------------------------------------------------------------------
Matchup: Houston vs. Longwood
Longwood wins

Match

In [79]:
#second round is listed, now needs formatting
for n in range(len(second_round)):
    second_round[n] = [(second_round[n][2*x],second_round[n][2*x+1]) for x in range(len(second_round[n])//2)]

In [80]:
#RUN SECOND ROUND
#try with 10 simulations
east_round_3 = []
west_round_3 = []
south_round_3 = []
midwest_round_3 = []
third_round = [east_round_3,west_round_3,south_round_3,midwest_round_3]

for division_number in range(len(second_round)):
    for match in second_round[division_number]:
        print('Matchup: {} vs. {}'.format(match[0], match[1]))
        res=matchup(match[0],match[1], reps=REPS)
        winners = np.mean([x[0]>x[1] for x in res])
        if winners == 0.5:
            res=matchup(match[0],match[1])
            winners=res[0][0]>res[0][1]
        if winners > 0.5:
            print('{} wins'.format(match[0]))
            third_round[division_number].append(match[0])
        else:
            print('{} wins'.format(match[1]))
            third_round[division_number].append(match[1])
        print()
    print('----------------------------------------------------------------')

Matchup: UConn vs. Fla. Atlantic
UConn wins

Matchup: San Diego St. vs. Yale
Yale wins

Matchup: BYU vs. Morehead St.
Morehead St. wins

Matchup: Drake vs. Iowa St.
Drake wins

----------------------------------------------------------------
Matchup: North Carolina vs. Mississippi St.
North Carolina wins

Matchup: Grand Canyon vs. Col. of Charleston
Col. of Charleston wins

Matchup: Clemson vs. Colgate
Clemson wins

Matchup: Dayton vs. Arizona
Dayton wins

----------------------------------------------------------------
Matchup: Longwood vs. Nebraska
Nebraska wins

Matchup: James Madison vs. Vermont
Vermont wins

Matchup: Texas Tech vs. Kentucky
Kentucky wins

Matchup: Boise St. vs. Marquette
Marquette wins

----------------------------------------------------------------
Matchup: Purdue vs. Utah St.
Purdue wins

Matchup: McNeese vs. Samford
McNeese wins

Matchup: Oregon vs. Creighton
Creighton wins

Matchup: Texas vs. Tennessee
Tennessee wins

-----------------------------------------

In [81]:
#third round is listed, now needs formatting
for n in range(len(third_round)):
    third_round[n] = [(third_round[n][2*x],third_round[n][2*x+1]) for x in range(len(third_round[n])//2)]

In [82]:
#RUN THIRD ROUND
#try with 10 simulations
east_round_4 = []
west_round_4 = []
south_round_4 = []
midwest_round_4 = []
fourth_round = [east_round_4,west_round_4,south_round_4,midwest_round_4]

for division_number in range(len(third_round)):
    for match in third_round[division_number]:
        print('Matchup: {} vs. {}'.format(match[0], match[1]))
        res=matchup(match[0],match[1], reps=REPS)
        winners = np.mean([x[0]>x[1] for x in res])
        if winners == 0.5:
            res=matchup(match[0],match[1])
            winners=res[0][0]>res[0][1]
        if winners > 0.5:
            print('{} wins'.format(match[0]))
            fourth_round[division_number].append(match[0])
        else:
            print('{} wins'.format(match[1]))
            fourth_round[division_number].append(match[1])
        print()
    print('------------------------------------------------------------------')

Matchup: UConn vs. Yale
UConn wins

Matchup: Morehead St. vs. Drake
Drake wins

------------------------------------------------------------------
Matchup: North Carolina vs. Col. of Charleston
Col. of Charleston wins

Matchup: Clemson vs. Dayton
Dayton wins

------------------------------------------------------------------
Matchup: Nebraska vs. Vermont
Vermont wins

Matchup: Kentucky vs. Marquette
Marquette wins

------------------------------------------------------------------
Matchup: Purdue vs. McNeese
McNeese wins

Matchup: Creighton vs. Tennessee
Creighton wins

------------------------------------------------------------------


In [83]:
final_four = []
for match in fourth_round:
    print('Matchup: {} vs. {}'.format(match[0], match[1]))
    res=matchup(match[0],match[1], reps=REPS)
    winners = np.mean([x[0]>x[1] for x in res])
    if winners == 0.5:
        res=matchup(match[0],match[1])
        winners=res[0][0]>res[0][1]
    if winners > 0.5:
        print('{} wins'.format(match[0]))
        final_four.append(match[0])
    else:
        print('{} wins'.format(match[1]))
        final_four.append(match[1])
    print()


Matchup: UConn vs. Drake
Drake wins

Matchup: Col. of Charleston vs. Dayton
Col. of Charleston wins

Matchup: Vermont vs. Marquette
Vermont wins

Matchup: McNeese vs. Creighton
Creighton wins



In [84]:
#FINAL FOUR
finals=[]
team_A = final_four[0]
team_B = final_four[1]
print('Matchup: {} vs. {}'.format(team_A, team_B))
res=matchup(team_A,team_B, reps=REPS)
winners = np.mean([x[0]>x[1] for x in res])
if winners == 0.5:
    res=matchup(team_A,team_B)
    winners=res[0][0]>res[0][1]
if winners > 0.5:
    print('{} wins'.format(team_A))
    finals.append(team_A)
else:
    print('{} wins'.format(team_B))
    finals.append(team_B)
print()

team_A = final_four[2]
team_B = final_four[3]
print('Matchup: {} vs. {}'.format(team_A, team_B))
res=matchup(team_A,team_B, reps=REPS)
winners = np.mean([x[0]>x[1] for x in res])
if winners == 0.5:
    res=matchup(team_A,team_B)
    winners=res[0][0]>res[0][1]
if winners > 0.5:
    print('{} wins'.format(team_A))
    finals.append(team_A)
else:
    print('{} wins'.format(team_B))
    finals.append(team_B)
print()

Matchup: Drake vs. Col. of Charleston
Drake wins

Matchup: Vermont vs. Creighton
Creighton wins



In [85]:
total_reps=REPS
total_ou = 0
print('Matchup: {} vs. {}'.format(finals[0], finals[1]))
res=matchup(match[0],match[1], reps=REPS)
winners = np.mean([x[0]>x[1] for x in res])
for score in res:
    total_ou += score[0] + score[1]
if winners == 0.5:
    res=matchup(match[0],match[1])
    total_ou += res[0][0] + res[0][1]
    total_reps+=1
    winners=res[0][0]>res[0][1]
if winners > 0.5:
    print('{} wins'.format(match[0]))
    final_four.append(match[0])
else:
    print('{} wins'.format(match[1]))
    final_four.append(match[1])
print()
print('Tiebreaker O/U: {}'.format(total_ou//total_reps))

Matchup: Drake vs. Creighton
Creighton wins

Tiebreaker O/U: 1600


In [91]:
total_ou//100

160